In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import cv2
import pathlib

# Importing Data

In [ ]:
training_data='/kaggle/input/brain-tumor-mri-dataset/Training'
testing_data='/kaggle/input/brain-tumor-mri-dataset/Testing'

In [ ]:
training_data=pathlib.Path(training_data)
testing_data=pathlib.Path(testing_data)

In [ ]:
len(list(training_data.glob('*/*')))

In [ ]:
len(list(testing_data.glob('*/*')))

In [ ]:
data_set_training={
       'glioma_tumor': list(training_data.glob('glioma/*')),
       'meningioma_tumor':list(training_data.glob('meningioma/*')),
       'no_tumor': list(training_data.glob('notumor/*')),
       'pituitary_tumor': list(training_data.glob('pituitary/*')),
}

In [ ]:
data_set_testing={
       'glioma_tumor': list(testing_data.glob('glioma/*')),
       'meningioma_tumor':list(testing_data.glob('meningioma/*')),
       'no_tumor': list(testing_data.glob('notumor/*')),
       'pituitary_tumor': list(testing_data.glob('pituitary/*')),
}

In [ ]:
data_set_index={
       'glioma_tumor': 0,
       'meningioma_tumor': 1,
       'no_tumor': 2,
       'pituitary_tumor': 3,
}

In [ ]:
images=[data_set_testing['glioma_tumor'][4],data_set_testing['meningioma_tumor'][0],data_set_testing['pituitary_tumor'][4],data_set_testing['no_tumor'][10]]
names=['glioma tumor','meningioma tumor','pituitary tumor','no tumor']
image_cv=[cv2.imread(str(i)) for i in images]
fig, axs = plt.subplots(1, len(names), figsize=(15, 5))
for i in range(len(names)):
    axs[i].imshow(image_cv[i])
    axs[i].set_title(names[i])
    axs[i].axis('off')

# Image Preprocessing

In [ ]:
cv2.imread(str(data_set_training['no_tumor'][0]))

In [ ]:
X=[]
y=[]
for tumor,images in data_set_training.items():
    for image in images:
        img=cv2.imread(str(image))
        X.append(cv2.resize(img,(180,180)))
        y.append(data_set_index[tumor])

for tumor,images in data_set_testing.items():
    for image in images:
        img=cv2.imread(str(image))
        X.append(cv2.resize(img,(180,180)))
        y.append(data_set_index[tumor])

X=np.array(X)
y=np.array(y)

In [ ]:
X.shape,y.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,random_state=0,test_size=0.2)

In [ ]:
X_train.shape,X_test.shape

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
X_train_scaled=X_train/255
X_test_scaled=X_test/255

In [ ]:
y_test_categorical=keras.utils.to_categorical(y_test)
y_train_categorical=keras.utils.to_categorical(y_train)

# Training Model

In [ ]:
model=keras.Sequential([
    
    layers.Conv2D(64,(3,3),activation='relu',input_shape=(180,180,3)),
    layers.MaxPool2D(2,2),
    layers.Dropout(0.3),
    
    layers.Conv2D(64,(3,3),activation='relu',padding='same'),
    layers.MaxPool2D(2,2), 
    layers.Dropout(0.3),
    
    layers.Conv2D(128,(3,3),activation='relu',padding='same'),
    layers.MaxPool2D(2,2), 
    layers.Dropout(0.4),
    
    layers.Flatten(),
    layers.Dense(128,activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(16,activation='relu'),
    layers.Dense(4,activation='softmax')
    
    
    
])

In [ ]:
model.compile( loss='categorical_crossentropy',
               optimizer='adam',
             metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history=model.fit(X_train_scaled,y_train_categorical,epochs=15,validation_split=0.1)

# Performance assessement 

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
epochs = range(len(acc))
fig = plt.figure(figsize=(12,7))
plt.plot(epochs,acc,'r',label="Training Accuracy")
plt.plot(epochs,val_acc,'b',label="Validation Accuracy")
plt.legend(loc='upper left')
plt.show()

In [ ]:
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(len(loss))
fig = plt.figure(figsize=(12,7))
plt.plot(epochs,loss,'r',label="Training loss")
plt.plot(epochs,val_loss,'b',label="Validation loss")
plt.legend(loc='upper left')
plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

In [ ]:
y_arr=model.predict(X_test_scaled)
y_arr[2]

In [ ]:
y_pred=[]
for i in y_arr:
    y_pred.append(np.argmax(i))
y_pred[2]

In [ ]:
sn.heatmap(confusion_matrix(y_test,y_pred),annot=True,fmt='d')
plt.ylabel('True')
plt.xlabel('Predicted')

In [ ]:
print(classification_report(y_test,y_pred))

In [ ]:
model.evaluate(X_test_scaled,y_test_categorical)

In [ ]:
model.save('braintumorModel.h5')